<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240406%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240406T062325Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8f3f107398d2e3c67277ed5515c066e2ee7794fde06df91d563b69616f4c086d9a480e848433b032c02c1281d0057abd4be820f195a876634e7263fe46e439134009a3cfa28669bac2d09b4c3de046458556a7372b890e3a8294ac9831d19b71e5cd4755f47a7caffa2a13c6e5e58991361e196c8206d0dd924835a4187491a2ce68625c69ba4dd027e6a282d00f1e11669a131b4aedc7a3617eef14727a7ac06e212c08aa836f6675d7b35bca3e2bb10c171eeb6654347ec22fca184746977d41be6c5649718d263696149cfc3cf54a2e7078be83e64a69d70c63c63cac889a08e984b4bc06c056a3e6aa8b1a2b3fa6a407690245f272d9512b2b6099e6b4a6,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240406%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240406T062325Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dde8a797418b2e765b1ae04a451d4786bc062e6bb076814d44b6e1051a2d792b6d0505d5749820804bf029a88f144aade0d3ffc7f68754795011a444f892d9960a42707c9d97263bc4c69adde7a29fc669d094df1ff445a59b74a66f911c59ff18fb1322ecb9fafaaaa5017c39e39c1ef940ec9fab18cdd738c8e91212f19f548480d6191e827a7c6349d422db720c6451f9c915c92ff7a542dd70f21e2f2183df5c6b1d168cb3180a82434067054d48c96b87a7f7f2d145ce5455ef84a607f4a04936e4f69b5605ac1e9104a053dd16fd71f9cea804c2f13d5c87ceb43f2b3004674bb510bcca2c9dffce05b14da3735260d90f8e27cbe25a89b600c348b3e59,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240406%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240406T062326Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D55f0fa2322a7cffca1d7225e50e25f572714339042214fd533e1c0305ed4089e4c398d9afbe4eee87d5f238147bd2ea92bb908461b517cc4b553b3e91f8ae04f232f0c56abe540266b7837113a83aaafe7ffaaf6735eec3fecbc9dcded2ed807f5175788aa1eac33cf3fb97f973c8a047762a22029cb40514955c1db42d5d86d3e0c6bad0819a930204735315904477c1b1ec0426ff343989ad0b7896a039ff40757156b5160e6e6f9c4836011adc00d2f181d46fd361dcaa21db1e0d871e80f00205ce0e6d7572f5197cd86dc1ba98f30093807e0647ae229cec1ecebccbbebac97cf4886bf8d4644fe2123cdecf0465b7d897a3f3c6914ce60eb5d0a69893f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powered by [Gemma 2B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the Python Enhancement Proposal ([PEPs](https://peps.python.org/)) documentation using a Retrival-Augmented Generation ([RAG](https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/)) sponsored by a Chroma vectorial database using the open source embeddings [hkunlp/instructor-large](https://huggingface.co/hkunlp/instructor-large) of 768 entries, orchested by [langchaing](https://python.langchain.com/docs/use_cases/chatbots/) that is a framework that let you use differen models and less code changes on the architecture when you decide to test other providers

## Requirements

In [ ]:
!pip install chromadb #!pip install chromadb==0.3.26
# !pip install ydata-profiling #!pip install ydata-profiling==4.6.1
!pip install langchain #!pip install langchain==0.0.345 ##!pip install langchain-core==0.1.31
#!pip install pydantic  #!pip install pydantic==1.10.14
!pip install sentence-transformers   #!pip install sentence-transformers==2.6.1
!pip install InstructorEmbedding  #!pip install InstructorEmbedding==1.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 11.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 81.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
#base
import os
import json
import re
import uuid
import time
import csv
import pandas as pd
import numpy as np
from typing import *
# variables
from kaggle_secrets import UserSecretsClient
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset
import requests
from bs4 import BeautifulSoup
# embeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
# vector database
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Classes

### Questions and answers data

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file
        answers_path (str): Path to the Answers CSV file
        tags_path (str): Path to the tags CSV file

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.tags_path = '../input/pythonquestions/Tags.csv'
        self.regex = r"<\/?[\w\s]*>"

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        df_questions = pd.read_csv(
            self.questions_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Score',
                'Title'
            ]
        )
        df_answers = pd.read_csv(
            self.answers_path,
            encoding="ISO-8859-1",
            usecols=[
                'ParentId',
                'Score',
                'Body'
            ]
        )
        df_tags = pd.read_csv(
            self.tags_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Tag'
            ]
        )
        return df_questions, df_answers, df_tags


    def qa_data(self):
        """Cleans, merges, and formats the question and answer data."""
        df_questions, df_answers, df_tags = self.load_data()
        # Rename
        df_questions.rename(
            columns={
                'Title': 'Question',
                'Score': 'question_score'
            },
            inplace=True
        )
        df_answers.rename(
            columns={
                'Body': 'Answer',
                'ParentId':'Id',
                'Score': 'answer_score'
            },
            inplace=True
        )
        # Filter by score
        df_questions = df_questions[df_questions['question_score'] > 5].copy()
        # Sort and deduplicate answers
        df_answers = df_answers.sort_values(
            'answer_score',
            ascending=False
        ).drop_duplicates(subset=['Id'])
        # Merge
        df_qa = df_questions.merge(
            df_answers,
            how='left',
            on='Id'
        ).merge(
            df_tags,
            how='left',
            on='Id'
        )
        # filter for python  questions
        df_qa = df_qa[df_qa['answer_score'] > 5].copy()
        df_qa = df_qa[df_qa['Tag']=='python'].copy()
        df_qa['Answer'] = df_qa['Answer'].apply(
            lambda x: re.sub(
                self.regex,
                "",
                x
            )
        )
        return df_qa

    def get_fine_tunning_data(self):
        """Returns a list of formatted user-assistant strings."""
        df_qa_data = self.qa_data()
        data = [
            f"<-change-of-interlocutor->user:\n{row['Question']}\n<-change-of-interlocutor->assistant:\n{row['Answer']}"
            for index, row
            in df_qa_data.iterrows()
        ]
        return data
    def get_qa_data(self):
        """Returns a list of records dictionaries """
        df_qa_data = self.qa_data()
        data = df_qa_data[
            [
                'Question',
                'Answer'
            ]
        ].to_dict(orient='records')
        return data


### Python Enhancement Proposals data

In [ ]:
class Peps:
    """
    Scrapes Python Enhancement Proposals (PEPs) from https://peps.python.org/
    """

    def __init__(self):
        """Initiates the scraping process."""
        self.base_url = 'https://peps.python.org/'

    def scraper(self, url):
        """
        Fetches the HTML content of a given URL.

        Args:
            url (str): The URL to fetch.

        Returns:
            BeautifulSoup: A BeautifulSoup object representing the parsed HTML.
        """
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')

    def _fetch_pep_links(self):
        """Fetches links to individual PEP pages (private method).

        Returns:
            list: A list of PEP URLs.
        """
        soup = self.scraper(self.base_url)
        return list(
            set(
                [
                    self.base_url + ref['href']
                    for ref in soup.find_all('a', class_='pep reference internal')
                ]
            )
        )

    def _download_peps(self):
        """Downloads the content of individual PEPs (private method).

        Returns:
            list: A list of BeautifulSoup objects representing individual PEPs.
        """
        pep_links = self._fetch_pep_links()
        return [self.scraper(pep_link) for pep_link in pep_links]

    def scrape(self):
        """Extracts the relevant text content from each PEP.

        Returns:
            list: A list of text strings, each representing the content of a PEP.
        """
        pep_soups = self._download_peps()
        return [
            '\n'.join([word.text.replace('"""',"'") for word in soup.find_all('section')])
            for soup in pep_soups
        ]

    def jsonl_format(self):
        """Format the spcraped data to a jsonl format is it alist of dictionaries
        Returns:
            list: A list of dictionaries with the following:
                page_content: The content of each section scraped
                source: The linke where it was scraped
        """
        source_links = self._fetch_pep_links()
        pep_content = self.scrape()
        pep_data = dict(zip(source_links,pep_content))
        return [
            {
                'text': value,
                'source': key
            }
            for key, value in pep_data.items()
        ]

### Embeddings

In [ ]:
class Embeder:
    """
    Creates text document embeddings (numerical representations) for semantic search,
    similarity comparison, and related natural language processing tasks. Offers the choice
    between a larger model for richer embeddings or a smaller, more efficient model.

    Attributes:
        large (bool): Controls the size of the embedding model. When set to True, uses a
                      larger model for richer embeddings (default: True).
        model (str): The name of the embedding model to use. Changes based on the value of 'large'.
        device (str): The device to use for computation. Options include "cpu" or, if available, "cuda"
                      for GPU acceleration (default: "cuda").
    """

    def __init__(self, large: bool = False):
        """
        Initializes the Embeder instance with preferences for model size.

        Args:
            large (bool, optional): If True, initializes with the larger embedding model.
                                    Defaults to False for the smaller, faster model.
        """
        self.large = large
        if self.large:
            self.model = "hkunlp/instructor-large"
            self.device = "cuda"  # If GPU is available
        else:
            self.model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

    def instructor(self):
        """
        Configures the embedding pipeline based on the selected model size.

        Returns:
           An embedding object from either the HuggingFaceInstructEmbeddings class
           (for the large model) or the SentenceTransformerEmbeddings class (for the smaller model).
        """
        if self.large:
            embed = HuggingFaceInstructEmbeddings(
                model_name=self.model,
                model_kwargs={"device": self.device}
            )
        else:
            embed = SentenceTransformerEmbeddings(model_name=self.model)
        return embed

    def run(self, docs_list: list):
        """
        Generates embeddings for a given list of text documents.

        Args:
            docs_list (list): A list of text documents to embed.

        Returns:
            List[List[float]]: A list of lists, where each inner list represents the
                               numerical embedding for a corresponding document in the input.
        """
        if docs_list is None:
            docs_list = ['']  # Prevent errors with empty input

        embed = self.instructor()  # Get the appropriate embedding object
        return embed.embed_documents(docs_list)


### Prompt

In [ ]:
prompt_config = [{
    "prompt_id": "0.0.1",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 0,
    "max_length": 250,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
},
{
    "prompt_id": "0.0.2",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than twice the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct answer
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 1,
    "max_length": 4096,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
}
{
    "prompt_id": "0.0.2",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than twice the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct answer
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 0,
    "max_length": 2048,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
}]

### Settings

In [ ]:
class Settings:
    """
    Manages and loads external configuration secrets for the application.  Utilizes a UserSecretsClient
    to securely retrieve sensitive configuration values.
    """

    def __init__(self):
        """
        Initializes the Settings class and retrieves configuration secrets.
        """
        self.user_secrets = UserSecretsClient()  # Create an instance for accessing secrets

        # Load individual secrets (add descriptions for clarity)
        self.CHUNK_SIZE = self.user_secrets.get_secret("CHUNK_SIZE")  # The size of data chunks for processing
        self.CHUNK_OVERLAP = self.user_secrets.get_secret("CHUNK_OVERLAP")  # Ooverlap between data chunks
        self.CHROMA_NAME_INDEX = self.user_secrets.get_secret("CHROMA_NAME_INDEX")  # Vectorial Database identifier
        self.K = self.user_secrets.get_secret("K")  # Could be a parameter for an algorithm
        self.NN_THRESHOLD = self.user_secrets.get_secret("NN_THRESHOLD")  # A threshold for a neural network or similarity metric
        self.PROMPT_ID = self.user_secrets.get_secret("PROMPT_ID")  # An identifier for a prompt (likely in a text-based task and configs)

        # Given that 'prompt_config' is teh previous source of prompt definitions:
        self.prompt = [prompt for prompt in prompt_config if prompt['prompt_id'] == self.PROMPT_ID][0]


### Retriever

In [ ]:
embeder = Embeder()
settings = Settings()
class Retriever:
    """
    Retrieves relevant text chunks from a Chroma vectorial database based on a query. Leverages external
    configuration settings (Settings) and an embedding model (Embeder).

    Handles text splitting, vector database loading, and querying.
    """

    def __init__(self):
        """
        Initializes the Retriever class. Configures the text splitter using settings and stores the
        index name for the Chroma database.
        """
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(settings.CHUNK_SIZE),
            length_function=len,
            chunk_overlap=int(settings.CHUNK_OVERLAP)
        )
        self.index_name = settings.CHROMA_NAME_INDEX
        self.k = int(settings.K)
        self.nn_threshold = float(settings.NN_THRESHOLD)

    def load(self, data: list[dict]):
        """
        Loads and preprocesses data for storage in the Chroma database.

        Args:
            data (list[dict]): A list of dictionaries, each containing text content and optional metadata.

        Returns:
            list[Document]: A list of Document objects formatted for Chroma.
        """
        documents = []
        for obj in data:
            page_content = obj.get("text", "")
            metadata = {
                "source": obj.get("source", "local")
            }
            documents.append(Document(page_content=page_content, metadata=metadata))
        return documents

    def set(self, data: list[dict]):
        """
        Updates the Chroma vectorial database with new data.

        Args:
            data (list[dict]): A list of dictionaries, each containing text content and optional metadata.
        """
        data = self.load(data)  # Preprocess data
        documents = self.text_splitter.split_documents(data)  # Split into chunks

        # Generate embeddings
        instructor = embeder.instructor()
        vector_db = Chroma.from_documents(
            documents=documents,
            embedding=instructor,
            persist_directory=self.index_name
        )
        vector_db.persist()  # Store the database

    def query(
            self,
            message: str,
            k: int = self.k,
            threshold: float = self.nn_threshold
    ):
        """
        Retrieves the most similar text chunks from the Chroma database based on a given query.

        Args:
            message (str): The query text.
            k (int, optional): The number of similar chunks to retrieve (default from settings).
            threshold (float, optional): The similarity threshold to filter results (default from settings).

        Returns:
            list[str]: A list of similar text chunks, filtered by the threshold.
        """
        # Load the Chroma database
        vectorial_db = Chroma(
            embedding_function=embeder.instructor(),
            persist_directory=self.index_name
        )

        # Perform the similarity search
        res = vectorial_db.similarity_search_with_score(message, k=k)

        # Filter and return results
        relevant_results = list(
            set(  # Remove duplicates
                [
                    vector[0].page_content
                    for vector in res
                    if vector[1] < threshold  # Apply similarity threshold
                ]
            )
        )
        return [
            result.replace('"""',"'") # cleaning posible docString retrived from documented code
            for result
            in relevant_results
        ]


In [ ]:
''.replace('"""',"'")

''

### Message

In [ ]:
settings = Settings()
class Message:
    """
    Represents a message within a conversational context. Stores information about the message's role,
    content, timestamp, and manages saving and loading conversation history.
    """

    def __init__(self, role: str, content: str, timestamp: str = int(time.time()), prompt_id: str = settings.PROMPT_ID):
        """
        Initializes a Message object.

        Args:
            role (str): Indicates the role of the sender (e.g., 'user', 'system').
            content (str): The text content of the message.
            timestamp (str, optional): A timestamp for the message (defaults to the current time).
            prompt_id (str, optional): Identifier relating to a specific prompt configuration (from settings).
        """

        self.reply_id = str(uuid.uuid4())  # Generate a unique ID for the message
        self.role = role
        self.content = content
        self.timestamp = timestamp
        self.file = 'data/history.json'  # File for storing conversation history

        # Ensure the history file exists
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))  # Create an empty file if it doesn't exist

    def reply(self):
        """
        Formats a basic reply message structure.

        Returns:
            dict: A dictionary containing the message's reply ID, role, content, and timestamp.
        """
        return {
            'reply_id': self.reply_id,
            'role': self.role,
            'content': self.content,
            'timestamp': self.timestamp
        }

    def system_reply(self):
        """
        Generates a system reply using the prompt configuration.

        Returns:
            dict: A dictionary containing the system reply, including ID, role, timestamp,
                  and text content derived from settings.
        """
        prompt = settings.prompt
        return {
            'reply_id': self.reply_id,
            'role': 'system',
            'content': prompt['system'],
            'timestamp': self.timestamp
        }

    def new_chat(self):
        """
        Starts a new chat by initializing the history file.

        Returns:
            list: A list with the initial system reply and the user's message.
        """
        init_chat = [self.system_reply(), self.reply()]
        json.dump(init_chat, open(self.file, 'w'))
        return init_chat

    def update(self):
        """
        Updates the conversation history file by appending the current reply.
        """
        dict_history = json.load(open(self.file))
        dict_history.append(self.reply())
        json.dump(dict_history, open(self.file, 'w'))

    def restart_history(self):
        """
        Clears the conversation history file.
        """
        json.dump([], open(self.file, 'w'))


### Model Gemma

In [ ]:
class Gemma:
    """
    Implements a conversational AI chatbot powered by Gemma large language model. Initializes the model,
    tokenizer, and prepares settings from a configuration.
    """

    def __init__(self):
        """
        Initializes the Gemma chatbot instance.

        Loads the language model and its corresponding tokenizer from the settings configuration.
        Prepares the model for use on a GPU (if available).
        """
        self.prompt = settings.prompt  # Load prompt settings

        # Load language model and tokenizer
        self.model_checkpoint = self.prompt['model']
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.gemma = AutoModelForCausalLM.from_pretrained(
            self.model_checkpoint,
            torch_dtype=torch.float16  # Use half-precision for efficiency (if supported)
        ).cuda()  # Move model to GPU (if available)

    def chat(self, context: str):
        """
        Generates a chatbot response based on the provided conversational context.

        Args:
            context (str): The conversational input text.

        Returns:
            str: The generated text response from the chatbot.
        """
        # Prepare input for the language model
        input_text = context
        input_ids = self.tokenizer(input_text, return_tensors="pt")
        input_ids = {
            k: v.to("cuda") for k, v in input_ids.items()  # Move tensors to GPU
        }

        # Generate a response with the language model
        outputs = self.gemma.generate(
            **input_ids,
            max_length=self.prompt['max_length']  # Control response length
        )

        # Decode and return the generated text
        return self.tokenizer.decode(outputs[0])


### Expert Python Agent/Assistant

In [ ]:
retriever = Retriever()
gemma = Gemma()

class Agent:
    """
    Implements a conversational AI agent that leverages a knowledge database for information retrieval
    and integrates with a generative language model (Gemma) for response generation. Manages conversation
    history and question-answering logic.
    """

    def __init__(self):
        """
        Initializes the Agent, loading prompt settings and ensuring the conversation history file exists.
        """
        self.prompt = settings.prompt
        self.file = 'data/history.json'
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))  # Create an empty file if necessary

        self.token = '\n<-change-of-interlocutor->'  # Token to separate speakers in the chat history

    def augmented_question(self, question: str):
        """
        Enhances the user's question with relevant technical documentation.

        Args:
            question (str): The user's original question.

        Returns:
            str: The question augmented with technical documentation (if found), otherwise the original question.
        """
        tech_docs = retriever.query(question)
        if len(tech_docs) > 0:
            docs = self.prompt['technical_documentation']
            tech_docs = '\n* '.join(tech_docs)
            docs = docs.format(technical_documentation=tech_docs)
            augmented_reply = f"""{question}{docs}"""
            return augmented_reply
        else:
            return question

    def memory(self, question: str):
        """
        Prepares the conversational context (memory) for the language model.

        Args:
            question (str): The user's current question.

        Returns:
            str: Formatted conversation history with a clear separation between speakers, ready
                 for input to the language model.
        """

        dict_history = json.load(open(self.file))
        message = Message(
            role='user',
            content=self.augmented_question(question)
        )

        if len(dict_history) > 0:
            message.update()  # Add the latest message to history
            full_chat = json.load(open(self.file))
        else:
            full_chat = message.new_chat()  # Start a new conversation

        return '\n'.join(
            [
                self.token + reply['role'] + ': ' + reply['content']
                for reply in full_chat
            ]
        ) + self.token + 'assistant:'

    def get_answer(self, full_response: str, question: str):
        """
        Extracts the relevant answer from the language model's generated response.

        Args:
            full_response (str): The complete response generated by the language model.
            question (str): The user's original question.

        Returns:
            str: The extracted answer.
        """

        answer_list = full_response.split(self.token)
        pos_list = [
            pos
            for pos, answer
            in enumerate(answer_list)
            if question in answer
        ]
        tokened_answer = answer_list[pos_list[0] + 1]
        answer = tokened_answer.split('assistant:')[1]
        message = Message(
            role='assistant',
            content=answer
        )
        message.update()  # Update conversation history
        return answer

    def chat(self, question: str, verbose: bool = False):
        """
        Manages the core interaction with the Agent.

        Args:
            question (str): The user's query.
            verbose (bool, optional): If True, prints the language model's full response (default: False).

        Returns:
            str: The Agent's answer to the user's question.
        """

        memory = self.memory(question)  # Build conversational context
        full_response = gemma.chat(context=memory)  # Generate response
        if verbose:
            print(full_response)
        answer = self.get_answer(full_response, question)  # Extract the answer
        return answer



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Data

### fine-tunning data

In [ ]:
%%time
PythonQAData = pythonQAData()
python_qa = PythonQAData.get_qa_data()
print(python_qa[1486])

{'Question': 'How can I split a file in python?', 'Answer': "This one splits a file up by newlines and writes it back out. You can change the delimiter easily. This can also handle uneven amounts as well, if you don't have a multiple of splitLen lines (20 in this example) in your input file.\n\nsplitLen = 20         # 20 lines per file\noutputBase = 'output' # output.1.txt, output.2.txt, etc.\n\n# This is shorthand and not friendly with memory\n# on very large files (Sean Cavanagh), but it works.\ninput = open('input.txt', 'r').read().split('\\n')\n\nat = 1\nfor lines in range(0, len(input), splitLen):\n    # First, get the list slice\n    outputData = input[lines:lines+splitLen]\n\n    # Now open the output file, join the new slice with newlines\n    # and write it out. Then close the file.\n    output = open(outputBase + str(at) + '.txt', 'w')\n    output.write('\\n'.join(outputData))\n    output.close()\n\n    # Increment the counter\n    at += 1\n\n"}
CPU times: user 16.2 s, sys: 1

In [ ]:
print(
'\n* Mean lenght of a python common question:',
sum([len(q['Question']) for q in python_qa])/len(python_qa),
'\n* Standard deviation of length a common Python question:',
np.std([len(q['Question']) for q in python_qa])
)


* Mean lenght of a python common question: 50.4238285354066 
* Standard deviation of length a common Python question: 18.168485523170148


In [ ]:
%%time
PythonQAData = pythonQAData()
python_context = PythonQAData.get_fine_tunning_data()
print(
    'Python Questions loaded:',
    len(python_context),
    '\n',
    '\nSample question-answer:\n',
    python_context[-1][:1000]
)

Python Questions loaded: 33313 
 
Sample question-answer:
 <-change-of-interlocutor->user:
Why can yield be indexed?
<-change-of-interlocutor->assistant:
You are not indexing. You are yielding a list; the expression yield[0] is really just the same as the following (but without a variable):

lst = [0]
yield lst


If you look at what next() returned you'd have gotten that list:

&gt;&gt;&gt; def gen1():
...   t = yield[0]
...   assert t
...   yield False
...
&gt;&gt;&gt; g = gen1()
&gt;&gt;&gt; next(g)
[0]


You don't have to have a space between yield and the [0], that's all.

The exception is caused by you trying to apply the subscription to the contained 0 integer:

&gt;&gt;&gt; [0]        # list with one element, the int value 0
[0]
&gt;&gt;&gt; [0][0]     # indexing the first element, so 0
0
&gt;&gt;&gt; [0][0][0]  # trying to index the 0
Traceback (most recent call last):
  File "&lt;stdin&gt;", line 1, in &lt;module&gt;
TypeError: 'int' object is not subscriptable


If you want t

## few-shot data

In [ ]:
few_shot_list=[7,6009,12,36,8034,130,141,537,1057,5042]
for i, q in enumerate(few_shot_list):
    print('_'*100,f'\n{i}.',python_qa[q])

____________________________________________________________________________________________________ 
0. {'Question': "How do I use Python's itertools.groupby()?", 'Answer': 'As Sebastjan said, you first have to sort your data. This is important.\n\nThe part I didn\'t get is that in the example construction\n\ngroups = []\nuniquekeys = []\nfor k, g in groupby(data, keyfunc):\n   groups.append(list(g))    # Store group iterator as a list\n   uniquekeys.append(k)\n\n\nk is the current grouping key, and g is an iterator that you can use to iterate over the group defined by that grouping key. In other words, the groupby iterator itself returns iterators.\n\nHere\'s an example of that, using clearer variable names:\n\nfrom itertools import groupby\n\nthings = [("animal", "bear"), ("animal", "duck"), ("plant", "cactus"), ("vehicle", "speed boat"), ("vehicle", "school bus")]\n\nfor key, group in groupby(things, lambda x: x[0]):\n    for thing in group:\n        print "A %s is a %s." % (thing[

## RAG data

In [ ]:
%%time
peps = Peps()
peps_corpus = peps.scrape()
print(
    'corpus lenght:',
    len(peps_corpus),
    '\nsection <n> lenght:',
    len(peps_corpus[0]),
    '\nsample text:\n\n',
    peps_corpus[10][1306:2000]
)

corpus lenght: 644 
section <n> lenght: 24494 
sample text:

 tly have an obscure
operating system dependent failure mode that allows them to return values that
are not, in fact, suitable for such operations.
This is due to some operating system kernels (most notably the Linux kernel)
permitting reads from /dev/urandom before the system random number
generator is fully initialized, whereas most other operating systems will
implicitly block on such reads until the random number generator is ready.
For the lower level os.urandom and random.SystemRandom APIs, this PEP
proposes changing such failures in Python 3.6 from the current silent,
hard to detect, and hard to debug, errors to easily detected and debugged errors
by raising BlockingIOError with 
CPU times: user 51.2 s, sys: 781 ms, total: 51.9 s
Wall time: 4min 27s


In [ ]:
%%time
peps = Peps()
data = peps.jsonl_format()
data[-1]['text'][:2000]

CPU times: user 52.6 s, sys: 687 ms, total: 53.3 s
Wall time: 1min 22s


'\n\nPython Enhancement Proposals\n\nPython » \nPEP Index » \nPEP 626\n\n\n\n\n\nToggle light / dark / auto colour theme\n\n\n\n\nPEP 626 – Precise line numbers for debugging and other tools.\n\nAuthor:\nMark Shannon <mark at hotpy.org>\nBDFL-Delegate:\nPablo Galindo <pablogsal at python.org>\nStatus:\nFinal\nType:\nStandards Track\nCreated:\n15-Jul-2020\nPython-Version:\n3.10\nPost-History:\n17-Jul-2020\n\n\n\nTable of Contents\nAbstract\nMotivation\nRationale\nSpecification\nTracing\nWhat is considered to be code for the purposes of tracing\nExample event sequences\nThe f_lineno attribute\nThe new co_lines() method of code objects\nZero width ranges\n\n\nThe co_linetable attribute\nThe co_lnotab attribute\n\n\nBackwards Compatibility\nExamples of code for which the sequence of trace events will change\npass statement in an if statement.\nMultiple pass statements.\n\n\nC API\nOut of process debuggers and profilers\n\n\nPerformance Implications\nReference Implementation\nCopyright\nRef

# RAG

In [ ]:
%%time
embeder = Embeder()
vec = embeder.run(["hello I'm goku"])[0]
print(len(vec))
vec[:10]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

384
CPU times: user 2.79 s, sys: 1.19 s, total: 3.98 s
Wall time: 19.1 s


[-0.1933707594871521,
 0.08631032705307007,
 0.2352217584848404,
 0.4100143313407898,
 -0.14344947040081024,
 0.19855117797851562,
 0.46806424856185913,
 -0.08672317862510681,
 -0.2187383770942688,
 0.1548473834991455]

In [ ]:
%%time
retriever = Retriever()
retriever.set(data)

CPU times: user 41min 9s, sys: 5min 7s, total: 46min 17s
Wall time: 40min 6s


In [ ]:
retriever = Retriever()
question = """
PEP 257
"""
retriever.query(question)

[]

In [ ]:
retriever = Retriever()
question = """
Rationale\nThis idea was proposed by Guido van Rossum on the python-ideas [1] mailing\nlist. The premise of his email was to slow the alteration of the  Python core\nsyntax, builtins and semantics to allow non-CPython implementations to catch\nup to the current state of Python, both 2.x and 3.x.\nPython, as a language is more than the core implementation –\nCPython – with a rich, mature and vibrant community of implementations, such\nas Jython [2], IronPython [3] and PyPy [4] that are a benefit not only to\nthe community, but to the language itself.\nStill others, such as Unladen Swallow [5] (a branch of CPython) seek not to\ncreate an alternative implementation, but rather they seek to enhance the\nperformance and implementation of CPython itself.\nPython 3.x was a large part of the last several years of Python’s\ndevelopment. Its release, as well as a bevy of changes to the language\nintroduced by it and the previous 2.6.x releases, puts alternative
"""
retriever.query(question)

['Rationale\nThis idea was proposed by Guido van Rossum on the python-ideas [1] mailing\nlist. The premise of his email was to slow the alteration of the  Python core\nsyntax, builtins and semantics to allow non-CPython implementations to catch\nup to the current state of Python, both 2.x and 3.x.\nPython, as a language is more than the core implementation –\nCPython – with a rich, mature and vibrant community of implementations, such\nas Jython [2], IronPython [3] and PyPy [4] that are a benefit not only to\nthe community, but to the language itself.\nStill others, such as Unladen Swallow [5] (a branch of CPython) seek not to\ncreate an alternative implementation, but rather they seek to enhance the\nperformance and implementation of CPython itself.\nPython 3.x was a large part of the last several years of Python’s\ndevelopment. Its release, as well as a bevy of changes to the language\nintroduced by it and the previous 2.6.x releases, puts alternative']

# Chatbot

## Loading few-shot to the memory

In [ ]:
file = 'data/history.json'
message = Message(
    role='sistem',
    content='foo'
)
init_chat = [message.system_reply()]
json.dump(init_chat, open(file, 'w'))
for i, q in enumerate(few_shot_list):
    sample = python_qa[q]
    mesage = Message(
        role='user',
        content=sample['Question']
    )
    reply = mesage.update()
    mesage = Message(
        role='assitant',
        content=sample['Answer']
    )
    reply = mesage.update()

history = json.load(open(file))
history

[{'reply_id': '50af5493-2d6c-4daa-9d49-ae0759d4b1df',
  'role': 'system',
  'content': '\nAct as a Python programming language expert assistant \nyour goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:\n1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary \n2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures\n3. Aim for plain language to ensure accessibility for all users\n4. If the question is off topic about programing lenguage you MUST to say "I\'m here to help you with Python programming language questions only, excuse me"\n5. You can say hello if you are greeted\n6. DO NOT repeat more than twice the same sentence!\n\nTAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION\n\nFor each question you answer perfectly, I will pay you US

### Testing the RAG system memory

In [ ]:
agent = Agent()
print(agent.memory("""
Version Control System
Currently the CPython and supporting repositories use Mercurial. As a modern distributed version control system, it has served us well since the migration from Subversion. However, when evaluating the VCS we must consider the capabilities of the VCS itself as well as the network effect and mindshare of the community around that VCS.

There are really only two real options for this, Mercurial and Git. The technical capabilities of the two systems are largely equivalent, therefore this PEP instead focuses on their social aspects.

It is not possible to get exact numbers for the number of projects or people which are using a particular VCS, however we can infer this by looking at several sources of information for what VCS projects are using.

The Open Hub (previously Ohloh) statistics [1] show that 37% of the repositories indexed by The Open Hub are using Git (second only to Subversion which has 48%) while Mercurial has just 2%,
"""))

# GPC - 0.0.1

In [ ]:
question

'\nRationale\nThis idea was proposed by Guido van Rossum on the python-ideas [1] mailing\nlist. The premise of his email was to slow the alteration of the  Python core\nsyntax, builtins and semantics to allow non-CPython implementations to catch\nup to the current state of Python, both 2.x and 3.x.\nPython, as a language is more than the core implementation –\nCPython – with a rich, mature and vibrant community of implementations, such\nas Jython [2], IronPython [3] and PyPy [4] that are a benefit not only to\nthe community, but to the language itself.\nStill others, such as Unladen Swallow [5] (a branch of CPython) seek not to\ncreate an alternative implementation, but rather they seek to enhance the\nperformance and implementation of CPython itself.\nPython 3.x was a large part of the last several years of Python’s\ndevelopment. Its release, as well as a bevy of changes to the language\nintroduced by it and the previous 2.6.x releases, puts alternative\n'

In [ ]:
%%time
agent = Agent()
question = 'what is the latest python version that is currently lunched'
print('*'*500,'\n final answer:',agent.chat(question, verbose=True))

<bos>
<-change-of-interlocutor->system: 
Act as a Python programming language expert assistant 
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary 
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than twice the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and corre

In [ ]:
%%time
agent = Agent()
print(agent.chat('what is python??'))

 Python is a programming language. It is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, script

In [ ]:
answer_list="""
Python is a programming language. It is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.

Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.
""".split('\n')
pos_list =  list(set(answer_list))
pos_list

['',
 'Python is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.',
 'Python is a programming language. It is a general-purpose, high-level, interpreted, object-oriented, and extensible language. It is used for web development, scripting, and system administration.']

In [ ]:
question

In [ ]:
%%time
agent = Agent()
print(agent.chat('my name is juan nice to meet you!!!'))

In [ ]:
history = json.load(open(file))
history

[{'reply_id': '50af5493-2d6c-4daa-9d49-ae0759d4b1df',
  'role': 'system',
  'content': '\nAct as a Python programming language expert assistant \nyour goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:\n1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary \n2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures\n3. Aim for plain language to ensure accessibility for all users\n4. If the question is off topic about programing lenguage you MUST to say "I\'m here to help you with Python programming language questions only, excuse me"\n5. You can say hello if you are greeted\n6. DO NOT repeat more than twice the same sentence!\n\nTAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION\n\nFor each question you answer perfectly, I will pay you US

# Fine-Tunning

## GPC - 0.1.0

In [ ]:
#for i in range(100000000000000000):
#    1*1